Greedy Solve
============

In [1]:
def compute_sorted_elements(stack):
    if len(stack)==0: return 0
    sorted_elements=1
    while(sorted_elements<len(stack) and stack[sorted_elements] <= stack[sorted_elements-1]):
        sorted_elements +=1
    
    return sorted_elements

def gvalue(stack):
    if len(stack)==0: return 100
    else: return stack[-1]
    

In [209]:
class Layout:
    def __init__(self, stacks, H):
        self.stacks = stacks
        self.sorted_elements = []
        self.sorted_stack = []
        self.unsorted_stacks = 0
        self.steps = 0
        self.current_step = 0
        self.moves = []
        self.H = H
        j=0
        
        for stack in stacks:
            self.sorted_elements.append(compute_sorted_elements(stack))
            if not self.is_sorted_stack(j):
                self.unsorted_stacks += 1
                self.sorted_stack.append(False)
            else: self.sorted_stack.append(True)
            j += 1
    
    def move(self,i,j, index=-1):
        c = self.stacks[i][index]
        if self.is_sorted_stack(i):
            self.sorted_elements[i] -= 1
            
        if self.is_sorted_stack(j) and gvalue(self.stacks[j]) >= c:
            self.sorted_elements[j] += 1
            
        self.stacks[i].pop(index)
        self.stacks[j].append(c)
        
        if index!=-1:  self.sorted_elements[i] = compute_sorted_elements(self.stacks[i])
        self.is_sorted_stack(i)
        self.is_sorted_stack(j)
        self.steps += 1
        self.current_step += 1
        self.moves.append((i,j,index))
        
        return c
        
    
        
    def highlighted_print(self, stack_set):
        for stack in self.stacks:
            if stack in stack_set:
                print('\033[94m'+str(stack)+'\033[0m')
            else:
                print(stack)
    
    def next(self, printed=False):
        if self.current_step == self.steps: 
            if printed: self.highlighted_print([])
            return
        i,j = self.moves[self.current_step]
        self.current_step += 1
        c=self.stacks[i].pop()
        self.stacks[j].append(c)
        
        if printed:
            s=[]
            if(self.current_step < self.steps):
                i,j = self.moves[self.current_step]
                s.append(self.stacks[i]), s.append(self.stacks[j])
            self.highlighted_print(s)
                

    def prev(self, printed=False):
        if self.current_step == 0:
            if printed: self.highlighted_print([])
            return
        
        self.current_step -= 1
        j,i = self.moves[self.current_step]
        c=self.stacks[i].pop()
        self.stacks[j].append(c)

        if printed:   
            s=[]
            if(self.current_step-1 > 0):
                i,j = self.moves[self.current_step-1]
                s.append(self.stacks[i]), s.append(self.stacks[j])
            self.highlighted_print(s)
        
    def is_sorted_stack(self, j):
        sorted = len(self.stacks[j]) == self.sorted_elements[j]

        if j<len(self.sorted_stack) and self.sorted_stack[j] != sorted: 
            self.sorted_stack[j] = sorted
            if sorted == True: self.unsorted_stacks -= 1
            else: self.unsorted_stacks += 1
        return sorted
    
    
        

In [210]:
def read_file(file, H):
    with open(file) as f:
        S, C = [int(x) for x in next(f).split()] # read first line
        stacks = []
        for line in f: # read rest of lines
            stack = [int(x) for x in line.split()[1::]]
            #if stack[0] == 0: stack.pop()
            stacks.append(stack)
            
        layout = Layout(stacks,H)
    return layout

In [381]:
def select_destination_stack(layout, orig, black_list=[], max_pos=100):
    s_o = layout.stacks[orig]
    c = s_o[-1]
    best_eval=-1000000;
    best_dest=None
    dest=-1;

    for dest in range(len(layout.stacks)):
            if orig==dest or dest in black_list: continue
            s_d = layout.stacks[dest]

            if(layout.H == len(s_d)): continue
            top_d=gvalue(s_d)

            ev=0

            if layout.is_sorted_stack(dest) and c<=top_d:
              #c can be well-placed: the sorted stack minimizing top_d is preferred.
              ev = 10000 - 100*top_d
            elif not layout.is_sorted_stack(dest) and c>=top_d:
              #unsorted stack with c>=top_d maximizing top_d is preferred
              ev = top_d
            elif layout.is_sorted_stack(dest):
              #sorted with minimal top_d
              ev = -100 - top_d
            else:
              #unsorted with minimal numer of auxiliary stacks
              ev = -10000 #+ required_stacks(dest)
            
            if layout.H - len(s_d) > max_pos:
              ev -= 10000

            if ev > best_eval:
                best_eval=ev
                best_dest=dest

    return best_dest

In [136]:
def reachable_height(layout, i):
    if not layout.is_sorted_stack(i): return -1;
    
    top = gvalue(layout.stacks[i])
    h = len(layout.stacks[i])
    if h==layout.H: return size;
    
    stack=layout.stacks[i]
    all_stacks = True #True: all the bad located tops can be placed in stack
    
    for k in range(len(layout.stacks)):
        if k==i: continue
        if layout.is_sorted_stack(k): continue
            
        stack_k=layout.stacks[k]
        unsorted = len(stack_k)-layout.sorted_elements[k]
        prev = 1000;
        for j in range (1,unsorted+1):
            if stack_k[-j] <= prev and stack_k[-j] <=top:
                h += 1
                if h==layout.H: return h
                prev = stack_k[-j]
            else: 
                if j==1: all_stacks=False
                break
                
    if all_stacks: return layout.H
    else: return h
             

In [350]:
def SF_move(layout, s_d=None):
    s_o = None
    s_d = None
    min_dif = 10000
    for i in range(len(layout.stacks)):
        if(layout.is_sorted_stack(i) and len(layout.stacks[i]) < layout.H):
            top = gvalue(layout.stacks[i])
            for k in range(len(layout.stacks)):
                if k!=i and not layout.is_sorted_stack(k):
                    if layout.stacks[k][-1] <= top and (top - layout.stacks[k][-1]) < min_dif:
                        min_dif = top - layout.stacks[k][-1]
                        s_d=i; s_o = k
    if s_o != None: 
        layout.move(s_o,s_d)
        return True
    return False

def SF_move_d(layout, s_d):
    s_o = None
    min_dif = 10000
    if(layout.is_sorted_stack(s_d) and len(layout.stacks[s_d]) < layout.H):
        top = gvalue(layout.stacks[s_d])
        for k in range(len(layout.stacks)):
            if k!=s_d and not layout.is_sorted_stack(k):
                if layout.stacks[k][-1] <= top and (top - layout.stacks[k][-1]) < min_dif:
                    min_dif = top - layout.stacks[k][-1]
                    s_o = k
    if s_o != None: 
        layout.move(s_o,s_d)
        return True
    return False


In [138]:
def SD_move(layout):
    best_ev = 0
    for i in range(len(layout.stacks)):
        prom = sum(layout.stacks[i]) / len(layout.stacks[i]) 
        ev = 10000 - 100*len(layout.stacks[i]) - prom
        if ev > best_ev:
            best_ev = ev
            s_o = i
            
    while len(layout.stacks[s_o])>0:
        s_d = select_destination_stack(layout,s_o)
        layout.move(s_o,s_d)
        if reachable_height(layout,s_o)==layout.H: return    

In [139]:
def greedy_solve(layout):
    while layout.unsorted_stacks>0:
        if not SF_move(layout):
            SD_move(layout)
        if layout.steps>1000: return 1000
    return layout.steps

In [9]:
def lazy_greedy(layout):
    while layout.unsorted_stacks>0:
        if not SF_move(layout): return

In [10]:
def solve_file(file,H):
    layout = read_file(file,H)
    greedy_solve(layout)
    return layout

Usando el Greedy_Solve
--------------------

Para resolver un problema basta con crear el layout como una lista de pilas: 

In [383]:
stacks=[[26,20,25,11,19,2,29,58,35,55],
[32,16,51,53,30,5,28,9,22,17],
[3,57,1,44,39,23,37,33,40,52],
[6,54,36,7,10,48,45,34,41,13],
[38,12,18,31,15,46,56,42,59,8],
[24,14,50,27,47,49,60,21,4,43]]

Luego creamos el layout pasando las *pilas* y la **altura máxima H**

In [384]:
layout = Layout(stacks, 12) #el numero es la altura máxima

Para resolver llamamos a la función greedy_solve la que nos retorna la cantidad de pasos que realizó

In [51]:
#greedy_solve(layout)

In [192]:
#Podemos ver el layout final
layout.stacks

[[26, 20, 25, 11, 19, 2, 29, 58, 35, 55, 5],
 [53],
 [3, 57, 1, 44, 39, 23, 37, 33, 40, 52, 22, 30],
 [6, 54, 36, 7, 10, 48, 45, 34, 41, 13, 16, 32],
 [38, 12, 18, 31, 15, 46, 56, 42, 59, 8, 9, 28],
 [24, 14, 50, 27, 47, 49, 60, 21, 4, 43, 17, 51]]

Stack Dismantling
---

Sacar elementos de manera normal tratando de dejarlos en buenas ubicaciones, es decir, privilegiar:
* posición correcta
* pilas invertidas

Además asegurarse que elementos mayores del stack puedan colocarse correctamente (++).
Esto se logra dejando los elementos mayores en posiciones top de otros stacks. La *condición* es la siguiente:
* Mayor elemento en top ([-1])
* Segundo mayor en [-1] o [-2]
* Etc..

Por lo tanto, si al decidir colocar un elemento mayor nos damos cuenta que no quedará en una posición conveniente, es válido realizar movimientos previos para que se cumpla la condición: i.e., se selecciona stack destino y se *llena* (movimientos GB no están permitidos). Se puede usar stack origen pero se considera un movimiento adicional (para mantener temporalmente el elemento mayor).

> Ojo que es posible que al colocar un elemento mayor, sea necesario bloquear algunas casillas superiores para forzar la condición.


Luego se van colocando los elementos *originales* en orden de vuelta al stack. Para ello posiblemente sea necesario realizar movimientos adicionales (pixie).
Si se puede colocar algún contenedor entremedio se hace.
Si nos damos cuenta que stack se puede llenar sin movimientos adicionales (aunque con contenedores distintos a los originales) volvemos a SF.

La función `fill_stack(s_d,n)` rellena el stack con `n` elementos. Puede usar elementos de cualquier stack con menor o igual altura. Si el stack está ordenado intenta mantener el orden o comenzar una pila invertida. Si stack desordenado coloca menor elemento mayor o igual al top.

In [162]:
def select_origin_stack(layout, dest, ori, rank):
    s_d = layout.stacks[dest]
    top_d = s_d[-1]
    best_eval=-1000000;
    best_orig=None
    orig=-1;

    for orig in range(len(layout.stacks)):
            if orig==dest or orig ==ori: continue
            s_o = layout.stacks[orig]

            if len(s_o)==0: continue           
            c=gvalue(s_o)
            if c in rank and rank[c] < layout.H-len(s_d): continue

            ev=0

            if layout.is_sorted_stack(dest) and c<=top_d:
                #c can be well-placed: the sorted stack maximizing c is preferred.
                ev = 10000 + 100*c
            elif not layout.is_sorted_stack(dest) and c>=top_d:
                #unsorted stack with c>=top_d minimizing c is preferred
                ev = -c
            else:
                ev = -100 - c 

            if ev > best_eval:
                best_eval=ev
                best_orig=orig

    return best_orig

In [163]:
def fill_stack(layout, s_d, n, ori, rank):
    for i in range(n):
        s_o = select_origin_stack(layout, s_d, ori, rank)
        layout.move(s_o,s_d)

def get_ranks(stack):
    r=1
    rank = {}
    for i in sorted(stack, reverse=True):
        rank[i] = r
        r += 1
    return rank

In [201]:
import numpy as np
def capacity(layout, s_o):
    capacity = 0
    h = np.zeros(len(layout.stacks))
    for i in range(len(layout.stacks)):
        if i==s_o: continue
        capacity += layout.H - len(layout.stacks[i])
        h[i] = layout.H
    return h.astype(int), capacity


def SDpp(layout, s_o, rank):
    hmax, capac = capacity(layout,s_o) # espacio libre
    ss_o = layout.stacks[s_o]
    while len(layout.stacks[s_o])>0:
        top = gvalue(ss_o)
        s_d = select_destination_stack (layout, s_o, max_pos=rank[top])
        pos = hmax[s_d]-len(layout.stacks[s_d])
        slack = capac-len(ss_o) # holgura
        if rank[top] < pos - slack:  # rellenar stack s_d
            #c=layout.stacks[s_o].pop(-1)
            fill_stack(layout, s_d, (pos - slack) - rank[top], s_o, rank)
            #layout.stacks[s_o].append(c)
        if rank[top] < pos: # se reduce máxima altura
            hmax[s_d] -= pos - rank[top]
            capac -= pos - rank[top]
        capac -= 1
        layout.move(s_o,s_d)

In [387]:
def SFpp(layout, s_d, rank, n=10):
    ub = 100
    cont = 0
    for r in rank:
        while True:
            (s, pos) = search_highest(layout, r, ub, s_d)
            if layout.sorted_elements[s] > len(layout.stacks[s])+pos: break #element is sorted
            c=force_move(layout, s, pos, s_d)
            ub=r
            cont+=1
            if cont == n: return
            if r==c: break
    while cont < n:
        SF_move_d(layout, s_d)
        cont += 1

In [388]:
def force_move(layout, s_o, pos, s_d): #pixie
    while pos<-1:
        s_tmp = select_destination_stack(layout, s_o, black_list=[s_d])
        layout.move(s_o,s_tmp)
        pos += 1
    return layout.move(s_o,s_d)

def search_highest(layout, c, ub, s_d):
    ret = None
    for pos in range(-1,-10,-1):
        max_c = 0
        for s in range(len(layout.stacks)):
            if s==s_d: continue
            ss = layout.stacks[s]
            if len(ss) < -pos: continue
            if ss[pos]>=c and ss[pos]<=ub and ss[pos]>max_c: 
                max_c=ss[pos]
                ret= (s,pos)
        if ret != None: return ret
    return None
            



In [401]:
s_o = 5
rank = get_ranks(layout.stacks[s_o])
SDpp(layout, s_o, rank)
layout.stacks

SFpp(layout, s_o, rank, 10)
layout.stacks

[[58, 55, 52, 35, 29, 26, 25, 20, 19, 17, 16, 10],
 [53, 51, 32, 30, 28, 22, 13, 11, 9, 8, 2],
 [57, 44, 40, 39, 37, 33, 23, 5, 3, 1],
 [54, 48, 45, 43, 41, 36, 34, 7, 6, 4],
 [59, 56, 46, 42, 38, 31, 18, 15, 12],
 [60, 50, 49, 47, 27, 24, 21, 14]]

In [403]:
len(layout.moves)

128

In [141]:
force_move(layout,5,-2,1)
layout.stacks

[[26, 20, 25, 11, 19, 2, 29, 58, 35, 55, 5, 30],
 [53, 51],
 [3, 57, 1, 44, 39, 23, 37, 33, 40, 52, 22, 32],
 [6, 54, 36, 7, 10, 48, 45, 34, 41, 13, 17, 16],
 [38, 12, 18, 31, 15, 46, 56, 42, 59, 8, 9, 28],
 [24, 14, 50, 27, 47, 49, 60, 21, 4, 43]]

In [65]:
lazy_greedy(layout)
layout.stacks

[[26, 20, 25, 11, 19, 2, 29, 58, 35, 55],
 [32, 16, 51, 53, 30, 5, 28, 9, 22],
 [3, 57, 1, 44, 39, 23, 37, 33, 40, 52],
 [6, 54, 36, 7, 10, 48, 45, 34, 41, 13, 17],
 [38, 12, 18, 31, 15, 46, 56, 42, 59, 8],
 [24, 14, 50, 27, 47, 49, 60, 21, 4, 43]]

[[58, 55, 52, 40, 33, 29, 26, 25, 20, 19, 11, 8],
 [53, 51, 43, 32, 28, 16, 9, 5, 4, 17, 2],
 [57, 44, 39, 37, 30, 23, 22, 21, 3, 1],
 [6, 54, 36, 7, 10, 48, 45, 34, 41, 13, 35],
 [38, 12, 18, 31, 15, 46, 56, 42, 59],
 [24, 14, 50, 27, 47, 49, 60]]

In [16]:
# lds() returns the length  
# of the longest decreasing 
# subsequence in arr[] of size n 
def lds(arr, n): 
  
    result = 0
    lis = [0 for i in range(n)] 
  
    # Initialize LIS values 
    # for all indexes  
    for i in range(n): 
        lis[i] = 1
  
    # Compute optimized LIS values  
    # in bottom up manner  
    for i in range(1, n): 
        for j in range(i): 
            if ( arr[i] < arr[j] and
                lis[i] < lis[j] + 1): 
                lis[i] = lis[j] + 1
             
    return lis

  

In [18]:
# Driver Code 
arr = layout.stacks[0]
n = len(arr) 
print(layout.stacks[0])
arr2 = lds(arr, n)
print(arr2, arr2.index(2)) 
min(max(arr2),5)


[26, 20, 25, 11, 19, 2, 29, 58, 35, 55]
[1, 2, 2, 3, 3, 4, 1, 1, 2, 2] 1


4

In [19]:
import numpy as np
# genera una secuencia de N elementos que deberían ser colocados en stack destino
def gen_sequence(arr, N):
    lds_arr = lds(arr, len(arr))
    T = min(max(lds_arr),N)
    
    first = lds_arr.index(T)
    print(lds_arr)
    while len(arr)-first-1 < N-T:
        T -= 1
        first = lds_arr.index(T)
    
    
    seq = []
    cindex=first
    while True:
        curr_index = cindex
        curr = lds_arr[cindex]
        seq.append(cindex)
        
        if curr==1: break
        
        cindex -= 1
        while lds_arr[cindex]!=curr-1 or arr[cindex]<arr[curr_index] : cindex -= 1
    
    print(seq)
            
    if N-T > 0:
        arr2 = np.array(arr[first+1:len(arr)])
        ind = arr2.argsort()[:(N-T)]
        ind = np.sort(ind)
        
        for el in ind:
            seq.insert(0,el+first+1)
        
   
    return seq
    

In [20]:
st = [32, 16, 51, 5]
seq=gen_sequence(st, 3)
print(seq)

[1, 2, 1, 3]
[3, 1, 0]
[3, 1, 0]


In [21]:
def sd_move2(layout, sd_o, sd_d, H):
    seq = gen_sequence( layout.stacks[sd_o], H-len( layout.stacks[sd_d]))
    print([layout.stacks[sd_o][x] for x in seq])
    for s in seq:
        layout.move(sd_o,sd_d,s)
    

In [35]:
stacks=[[26,20,25,11,19,2,29,58,35,55],
[32,16,51,53,30,5,28,9,22,17],
[3,57,1,44,39,23,37,33,40,52],
[6,54,36,7,10,48,45,34,41,13],
[38,12,18,31,15,46,56,42,59,8],
[24,14,50,27,47,49,60,21,4,43]]
layout = Layout(stacks, 12) #el numero es la altura máxima

In [36]:
layout.stacks

[[26, 20, 25, 11, 19, 2, 29, 58, 35, 55],
 [32, 16, 51, 53, 30, 5, 28, 9, 22, 17],
 [3, 57, 1, 44, 39, 23, 37, 33, 40, 52],
 [6, 54, 36, 7, 10, 48, 45, 34, 41, 13],
 [38, 12, 18, 31, 15, 46, 56, 42, 59, 8],
 [24, 14, 50, 27, 47, 49, 60, 21, 4, 43]]

In [46]:
greedy2(layout)

4
[0 3 2 0 0 6]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 9]
[5, 4, 3, 2, 1, 0]
[38, 42, 46, 47, 49, 56]
[1, 2, 3, 4, 3]
[2, 1, 0]
[15, 27, 31]
[1, 1]
[0]
[22, 12]
[58, 55, 52, 43, 35, 29, 26, 25, 21, 20, 19, 11]
[53, 51, 32, 30, 28, 16, 10, 9, 5, 15, 27, 31]
[57, 44, 40, 39, 37, 33, 17, 13, 3, 1, 22, 12]
[54, 48, 45, 41, 36, 34, 23, 8, 7, 6, 4, 2]
[]
[60, 59, 50, 24, 18, 14, 38, 42, 46, 47, 49, 56]


[[58, 55, 52, 43, 35, 29, 26, 25, 21, 20, 19, 11],
 [53, 51, 32, 30, 28, 16, 10, 9, 5],
 [57, 44, 40, 39, 37, 33, 17, 13, 3, 1],
 [54, 48, 45, 41, 36, 34, 23, 8, 7, 6, 4, 2],
 [56, 49, 47, 46, 42, 38, 31, 27, 22, 15],
 [60, 59, 50, 24, 18, 14, 12]]

In [49]:
layout.unsorted_stacks,layout.steps

(0, 165)

In [38]:
def greedy2(layout):

    ind = np.argsort(layout.sorted_elements)
    i = 0
    while layout.is_sorted_stack(ind[i]):
        i+=1
        
    s_o = ind[i]
    print(s_o)
    
    
    C = np.zeros(len(layout.stacks), int)
    for i in range(len(layout.stacks)):
        if i!=s_o:
            C[i] = layout.H-len(layout.stacks[i])
            
    stacks = (-C).argsort()
    print(C)

    for s_d in stacks:
        if C[s_d] >0:
            sd_move2(layout, s_o, s_d ,layout.H)

    for s in layout.stacks:
        print(s)
        
    lazy_greedy(layout)
    return layout.stacks

In [624]:
# Y también retroceder hasta el comienzo para poder ver los paso uno a uno
while layout.current_step-1 >0: 
    layout.prev()

In [16]:
#Luego de quedar al comienzo, cada vez que ejecutamos esto avanzamos un paso
layout.next(True)

NameError: name 'layout' is not defined